<a href="https://colab.research.google.com/github/KendallScott/Capstone_alt/blob/main/Data_Cleanup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

References:
https://geemap.org/notebooks/114_dynamic_world/


# Imports and Setup

In [1]:
!pip -q install --upgrade folium
!pip -q install geopandas
!pip -q install geojson
!pip -q install eeconvert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 17.7 MB/s eta 0:00:00


In [2]:
# reminder that if you are installing libraries in a Google Colab instance you will be prompted to restart your kernal

try:
    import geemap, ee
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        print("package not found, installing w/ pip in Google Colab...")
        !pip install geemap
    else:
        print("package not found, installing w/ conda...")
        !conda install mamba -c conda-forge -y
        !mamba install geemap -c conda-forge -y
    import geemap, ee

package not found, installing w/ pip in Google Colab...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.9 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11125 sha256=f426a3e518098bda1cf69357e0fb595ed3b8ed6523a546c075ad119822517d28
  Stored in directory: /root/.c

In [3]:
pip install eemont

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for eemont: filename=eemont-0.3.6-py3-none-any.whl size=135742 sha256=2140ae069d3dcf35bdd49c37b3b5533d7032132358841f2025e18cbcc7d9852f
  Stored in directory: /root/.cache/pip/wheels/c2/79/2c/4fed17c3d3b466bbf4fe5872eec11f189147043b01152a4f75
  Created wheel for ee_extra: filename=ee_extra-0.0.15-py3-none-any.whl size=236755 sha256=08f09a4a4aab2c6fafd9411320153115839098a992c08d27d013cfc13dd1db6a
  Stored in directory: /root/.cache/pip/wheels/29/96/0e/4e36b0dfd85e16867723df739294c0aa45a65b191adac4d959
Successfully built eemont ee_extra


In [4]:
# Standard imports
import requests
import json

import pandas as pd
import numpy as np
from PIL import Image

# Geospatial processing packages
import geopandas as gpd
import geojson

import shapely
from shapely.geometry import box

# Mapping and plotting libraries
import matplotlib.pyplot as plt
import matplotlib.colors as cl
import ee
import eeconvert as eec
import geemap
import folium
import eemont

import geopandas as gpd
from shapely.geometry import shape

### Mount Drive

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Authenticate Google Earth Engine
Make sure you have signed up for access to Google Earth Engine at https://signup.earthengine.google.com/#!/. Once your request has been approved, you should be able to access Google Earth Engine at https://code.earthengine.google.com/.

In [6]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=U2rZWdPWrvsXMcysogATp3KaAYvYi2RKM95M5IsKB-U&tc=55h5yPgrnrqWeMbe0ilLkLArPsyao-QPfr7U3SwhAI8&cc=IwV3uTO2GsmkTY4sWPg14nDJXLaXYa90khxkzqo823k

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1Adeu5BWsmTrBbvwHoRbkZD9H6zWYTxEB7iJJcBpEofZgPX2GT3xTKTyqpsg

Successfully saved authorization token.


In [7]:

# Read the GeoJSON file
mines = gpd.read_file('/content/drive/MyDrive/KY_Released_mines.geojson')

# Read data using GeoPandas
print("Data dimensions: {}".format(mines.shape))

Data dimensions: (12606, 19)


In [8]:
mines.sample(10)

,id,ACT_INAC,Calc_Acres,Contact,DATE_ISS,FeatCLS,MINE_STATU,National_I,ORIG_PERM,PERM_ACT,PER_NAME,PER_TYPE,PermitNo,QUAD_DESC,REGION_DES,Shape_Leng,Source,Type_Flag,geometry
5352,00000000000000002301,Released,10.973465,KY Division of Mine Permits,394786800000,SF,RC,KY0580089,0580089,RV 05/24/1985 NW 07/06/1982,B & P COAL CO,INTERIM,0580089,SITKA,PRESTONSBURG,7262.916359,KGS Project,INACT,"POLYGON ((-82.81873 37.92331, -82.81889 37.923..."
374,00000000000000000bc4,Released,53.472209,KY Division of Mine Permits,1176447600000,SF,FF,KY8480263,8480263,MT 1 06/03/2009 MI 1 03/10/2008 NW 1...,"COMBINED ENTERPRISES, LLC",PERMANENT,8480263,HARLAN,MIDDLESBORO,20196.004224,KYDMP,INACT,"POLYGON ((-83.26351 36.78158, -83.26323 36.781..."
11259,000000000000000019c8,Released,2.378573,KY Division of Mine Permits,852710400000,SF,RC,KY8984053,8984053,MI 07/10/1998 MT 11/01/1997 MI ...,MONEY BRANCH COAL COMPANY,PERMANENT,8984053,WILLIAMSON,PIKEVILLE,2212.521696,KYDMP,INACT,"POLYGON ((-82.30678 37.67097, -82.30621 37.671..."
2766,00000000000000002f68,Released,1.467472,KY Division of Mine Permits,275814000000,SF,RC,KY2985045,2985045,NW 09/28/1978,BELFRY COAL CORPORATION,INTERIM,2985045,WILSON,PIKEVILLE,1961.359964,KGS Project,TRNS,"POLYGON ((-82.35352 37.69216, -82.35351 37.691..."
6243,00000000000000002288,Released,2.477360,KY Division of Mine Permits,389862000000,SF,RC,KY0485200,0485200,NW 05/10/1982,MOUNTAIN SPUR COALS & ENERGY INC,INTERIM,0485200,HARLAN,MIDDLESBORO,2772.673563,KGS Project,INACT,"MULTIPOLYGON (((-83.36253 36.81749, -83.36250 ..."
6183,00000000000000000c19,Released,2.698745,KY Division of Mine Permits,514195200000,SF,RC,KY8485109,8485109,MT 01/06/1993 RN 03/25/1991 NW ...,SANDLICK COAL COMPANY INC,PERMANENT,8485109,HARLAN,MIDDLESBORO,1595.998771,KGS Project,INACT,"POLYGON ((-83.34973 36.82184, -83.34967 36.821..."
11707,000000000000000002e2,Released,19.623116,KY Division of Mine Permits,238921200000,SF,RC,KY590876X,590876X,NW 07/28/1977,GAR VAN COAL CO,PRE LAW,590876X,BARBOURVILLE,MIDDLESBORO,6669.594687,KGS Project,INACT,"POLYGON ((-83.94923 36.78043, -83.94907 36.780..."
1827,0000000000000000149d,Released,141.940018,KY Division of Mine Permits,614847600000,SF,RC,KY8770107,8770107,SU 03/20/1991 NW 06/26/1989,BELFRY COAL CORPORATION,PERMANENT,8770107,DAVID,PRESTONSBURG,35017.919373,KGS Project,TRNS,"MULTIPOLYGON (((-82.94629 37.62680, -82.94630 ..."
8439,00000000000000001f7d,Released,5.693176,KY Division of Mine Permits,328431600000,SF,RC,KY0135004,0135004,RV 04/30/1987 NW 05/29/1980,BARWICK COAL COMPANY INC,INTERIM,0135004,KRYPTON,LONDON,3346.292754,KGS Project,INACT,"POLYGON ((-83.36552 37.36788, -83.36548 37.367..."
11981,00000000000000002f5c,Released,6.911616,KY Division of Mine Permits,300610800000,SF,RC,KY2985028,2985028,NW 07/12/1979,RITA COAL CO,INTERIM,2985028,ELKHORN CITY,PIKEVILLE,13692.709260,KGS Project,INACT,"MULTIPOLYGON (((-82.35127 37.33603, -82.35062 ..."


In [9]:


# Get the shape geometry
region = eec.gdfToFc(mines)

In [10]:
mines_exploded= mines.explode()
mines_exploded

<ipython-input-10-86b5dc24b419>:1: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  mines_exploded= mines.explode()


id  ACT_INAC  Calc_Acres  \
0     0  00000000000000001111  Released  294.302954   
1     0  00000000000000001112  Released  305.739886   
      1  00000000000000001112  Released  305.739886   
      2  00000000000000001112  Released  305.739886   
      3  00000000000000001112  Released  305.739886   
...                       ...       ...         ...   
12603 2  00000000000000000472  Released   56.660359   
12604 0  0000000000000000308a  Released   14.972560   
      1  0000000000000000308a  Released   14.972560   
      2  0000000000000000308a  Released   14.972560   
12605 0  000000000000000004fc  Released   35.631015   

                             Contact       DATE_ISS FeatCLS MINE_STATU  \
0     0  KY Division of Mine Permits  1141718400000      SF         FF   
1     0  KY Division of Mine Permits  1141718400000      SF         FF   
      1  KY Division of Mine Permits  1141718400000      SF         FF   
      2  KY Division of Mine Permits  1141718400000      SF         FF   
      3  KY Division of Mine Permits  1141718400000      SF         FF   
...                              ...            ...     ...        ...   
12603 2  KY Division of Mine Permits   255254400000      SF         XX   
12604 0  KY Division of Mine Permits   166863600000      SF         XX   
      1  KY Division of Mine Permits   166863600000      SF         XX   
      2  KY Division of Mine Permits   166863600000      SF         XX   
12605 0  KY Division of Mine Permits   262598400000      SF         XX   

        National_I ORIG_PERM  \
0     0  KY8640180   8640180   
1     0  KY8640180   8640180   
      1  KY8640180   8640180   
      2  KY8640180   8640180   
      3  KY8640180   8640180   
...            ...       ...   
12603 2  KY686977X   686977X   
12604 0  KY388574X   388574X   
      1  KY388574X   388574X   
      2  KY388574X   388574X   
12605 0  KY710477X   710477X   

                                                  PERM_ACT  \
0     0  SU      03/07/2013  AM   1  05/12/2011  RN   1...   
1     0  SU      03/07/2013  AM   1  05/12/2011  RN   1...   
      1  SU      03/07/2013  AM   1  05/12/2011  RN   1...   
      2  SU      03/07/2013  AM   1  05/12/2011  RN   1...   
      3  SU      03/07/2013  AM   1  05/12/2011  RN   1...   
...                                                    ...   
12603 2                                 NW      02/02/1978   
12604 0             AM      07/14/1975  NW      04/16/1975   
      1             AM      07/14/1975  NW      04/16/1975   
      2             AM      07/14/1975  NW      04/16/1975   
12605 0                                 NW      04/28/1978   

                        PER_NAME   PER_TYPE PermitNo       QUAD_DESC  \
0     0         B & C ENERGY INC  PERMANENT  8640180           MAZIE   
1     0         B & C ENERGY INC  PERMANENT  8640180           MAZIE   
      1         B & C ENERGY INC  PERMANENT  8640180           MAZIE   
      2         B & C ENERGY INC  PERMANENT  8640180           MAZIE   
      3         B & C ENERGY INC  PERMANENT  8640180           MAZIE   
...                          ...        ...      ...             ...   
12603 2         CAL GLO COAL INC    PRE LAW  686977X    BARBOURVILLE   
12604 0   KY PACIFIC MINING CORP    PRE LAW  388574X    BARBOURVILLE   
      1   KY PACIFIC MINING CORP    PRE LAW  388574X    BARBOURVILLE   
      2   KY PACIFIC MINING CORP    PRE LAW  388574X    BARBOURVILLE   
12605 0  T & T DARBY COAL CO INC    PRE LAW  710477X  PENNINGTON GAP   

           REGION_DES    Shape_Leng       Source Type_Flag  \
0     0  PRESTONSBURG  53267.493940         CHIA      TRNS   
1     0  PRESTONSBURG  92560.008796        KYDMP      TRNS   
      1  PRESTONSBURG  92560.008796        KYDMP      TRNS   
      2  PRESTONSBURG  92560.008796        KYDMP      TRNS   
      3  PRESTONSBURG  92560.008796        KYDMP      TRNS   
...               ...           ...          ...       ...   
12603 2   MIDDLESBORO  16743.038634  KGS Project

In [11]:
# Get the shape geometry for Tennesssee mines
region = eec.gdfToFc(mines)

In [12]:
df = pd.DataFrame()

In [13]:
mines_exploded=mines.explode()
mines_exploded

<ipython-input-13-9f8c0177ef78>:1: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  mines_exploded=mines.explode()


id  ACT_INAC  Calc_Acres  \
0     0  00000000000000001111  Released  294.302954   
1     0  00000000000000001112  Released  305.739886   
      1  00000000000000001112  Released  305.739886   
      2  00000000000000001112  Released  305.739886   
      3  00000000000000001112  Released  305.739886   
...                       ...       ...         ...   
12603 2  00000000000000000472  Released   56.660359   
12604 0  0000000000000000308a  Released   14.972560   
      1  0000000000000000308a  Released   14.972560   
      2  0000000000000000308a  Released   14.972560   
12605 0  000000000000000004fc  Released   35.631015   

                             Contact       DATE_ISS FeatCLS MINE_STATU  \
0     0  KY Division of Mine Permits  1141718400000      SF         FF   
1     0  KY Division of Mine Permits  1141718400000      SF         FF   
      1  KY Division of Mine Permits  1141718400000      SF         FF   
      2  KY Division of Mine Permits  1141718400000      SF         FF   
      3  KY Division of Mine Permits  1141718400000      SF         FF   
...                              ...            ...     ...        ...   
12603 2  KY Division of Mine Permits   255254400000      SF         XX   
12604 0  KY Division of Mine Permits   166863600000      SF         XX   
      1  KY Division of Mine Permits   166863600000      SF         XX   
      2  KY Division of Mine Permits   166863600000      SF         XX   
12605 0  KY Division of Mine Permits   262598400000      SF         XX   

        National_I ORIG_PERM  \
0     0  KY8640180   8640180   
1     0  KY8640180   8640180   
      1  KY8640180   8640180   
      2  KY8640180   8640180   
      3  KY8640180   8640180   
...            ...       ...   
12603 2  KY686977X   686977X   
12604 0  KY388574X   388574X   
      1  KY388574X   388574X   
      2  KY388574X   388574X   
12605 0  KY710477X   710477X   

                                                  PERM_ACT  \
0     0  SU      03/07/2013  AM   1  05/12/2011  RN   1...   
1     0  SU      03/07/2013  AM   1  05/12/2011  RN   1...   
      1  SU      03/07/2013  AM   1  05/12/2011  RN   1...   
      2  SU      03/07/2013  AM   1  05/12/2011  RN   1...   
      3  SU      03/07/2013  AM   1  05/12/2011  RN   1...   
...                                                    ...   
12603 2                                 NW      02/02/1978   
12604 0             AM      07/14/1975  NW      04/16/1975   
      1             AM      07/14/1975  NW      04/16/1975   
      2             AM      07/14/1975  NW      04/16/1975   
12605 0                                 NW      04/28/1978   

                        PER_NAME   PER_TYPE PermitNo       QUAD_DESC  \
0     0         B & C ENERGY INC  PERMANENT  8640180           MAZIE   
1     0         B & C ENERGY INC  PERMANENT  8640180           MAZIE   
      1         B & C ENERGY INC  PERMANENT  8640180           MAZIE   
      2         B & C ENERGY INC  PERMANENT  8640180           MAZIE   
      3         B & C ENERGY INC  PERMANENT  8640180           MAZIE   
...                          ...        ...      ...             ...   
12603 2         CAL GLO COAL INC    PRE LAW  686977X    BARBOURVILLE   
12604 0   KY PACIFIC MINING CORP    PRE LAW  388574X    BARBOURVILLE   
      1   KY PACIFIC MINING CORP    PRE LAW  388574X    BARBOURVILLE   
      2   KY PACIFIC MINING CORP    PRE LAW  388574X    BARBOURVILLE   
12605 0  T & T DARBY COAL CO INC    PRE LAW  710477X  PENNINGTON GAP   

           REGION_DES    Shape_Leng       Source Type_Flag  \
0     0  PRESTONSBURG  53267.493940         CHIA      TRNS   
1     0  PRESTONSBURG  92560.008796        KYDMP      TRNS   
      1  PRESTONSBURG  92560.008796        KYDMP      TRNS   
      2  PRESTONSBURG  92560.008796        KYDMP      TRNS   
      3  PRESTONSBURG  92560.008796        KYDMP      TRNS   
...               ...           ...          ...       ...   
12603 2   MIDDLESBORO  16743.038634  KGS Project

In [ ]:
df = pd.DataFrame()
years = ['2023-07']
i=2

i_date = years[0]+'-01' # Initial date of interest (inclusive)
f_date = years[0]+'-31' # Final date of interest (exclusive)

while i<len(mines_exploded):
      fil_mines = mines_exploded.iloc[[i]]
      region = eec.gdfToFc(fil_mines)
      dc = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterBounds(region).filterDate(i_date, f_date) # Get dynamic world image collection by asset ID
      polygon = mines_exploded['geometry'].values[i]
      if polygon.geom_type == 'Polygon': # Extract the coordinates from the polygon,
        coordinates = polygon.exterior.coords[:]
        polygon = ee.Geometry.MultiPolygon(coordinates) # Create an ee.Geometry object from the coordinates
        max_error = 1 # Set the maxError parameter
        projected_polygon = polygon.transform('EPSG:4326', max_error) # Project the geometry coordinates
        ac = dc.getRegion(projected_polygon, scale=200 ).getInfo() #decrease scale to get more rows of data/granularity      df_constructor =pd.DataFrame(ac)

        df_constructor =pd.DataFrame(ac)
        df_constructor.columns = df_constructor.iloc[0]
        df_constructor = df_constructor.tail(-1)
        df_constructor['Company'] = mines_exploded['PER_NAME'].values[i]
        df = df.append(df_constructor)

      elif polygon.geom_type == 'LineString':
        coordinates = polygon.coords[:]
        polygon = ee.Geometry.LineString(coordinates)
        ac = dc.getRegion(polygon, scale=200 ).getInfo() #decrease scale to get more rows of data/granularity      df_constructor =pd.DataFrame(ac)

        df_constructor =pd.DataFrame(ac)
        df_constructor.columns = df_constructor.iloc[0]
        df_constructor = df_constructor.tail(-1)
        df_constructor['Company'] = mines_exploded['PER_NAME'].values[i]
        df_constructor['mine_id'] = mines_exploded['id'].values[i]
        df_constructor['orig_perm_id'] = mines_exploded['ORIG_PERM'].values[i]
        df = df.append(df_constructor)

      print(mines_exploded['PER_NAME'].values[i])
      i+=1

df =df.drop_duplicates()
df = df[df['label'].notna()]

file_name = 'ky_'+i_date+f_date
from google.colab import files
df.to_csv(file_name, encoding = 'utf-8-sig')
files.download(file_name)

<ipython-input-23-fa1f96d40843>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_constructor)


B & C ENERGY INC


<ipython-input-23-fa1f96d40843>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_constructor)


B & C ENERGY INC


<ipython-input-23-fa1f96d40843>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_constructor)


...

<ipython-input-23-fa1f96d40843>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_constructor)


HALL & HYLTON MINING COMPANY


<ipython-input-23-fa1f96d40843>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_constructor)


T COLEMAN ENERGY INC


<ipython-input-23-fa1f96d40843>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_constructor)


In [22]:
df

,id,longitude,latitude,time,water,trees,grass,flooded_vegetation,crops,shrub_and_scrub,built,bare,snow_and_ice,label,Company
1,20230705T160829_20230705T162114_T17SLC,-82.962111,38.040957,1688574175051,None,None,None,None,None,None,None,None,None,None,B & C ENERGY INC
2,20230710T160831_20230710T161853_T17SLC,-82.962111,38.040957,1689006174912,None,None,None,None,None,None,None,None,None,None,B & C ENERGY INC
3,20230718T161839_20230718T162101_T17SLC,-82.962111,38.040957,1689697971141,0.032271,0.742175,0.033115,0.028178,0.029994,0.033221,0.02904,0.03613,0.035817,1,B & C ENERGY INC
4,20230723T161831_20230723T162339_T17SLC,-82.962111,38.040957,1690129970031,None,None,None,None,None,None,None,None,None,None,B & C ENERGY INC
5,20230725T160829_20230725T162050_T17SLC,-82.962111,38.040957,1690302175614,None,None,None,None,None,None,None,None,None,None,B & C ENERGY INC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,20230710T160831_20230710T161853_T17SLB,-82.611768,37.742717,1689006189321,None,None,None,None,None,None,None,None,None,None,ENOXY COAL INC
2,20230718T161839_20230718T162101_T17SLB,-82.611768,37.742717,1689697985617,0.033791,0.704108,0.040599,0.032321,0.061391,0.03001,0.027825,0.029238,0.034854,1,ENOXY COAL INC
3,20230723T161831_20230723T162339_T17SLB,-82.611768,37.742717,1690129984504,None,None,None,None,None,None,None,None,None,None,ENOXY COAL INC
4,20230725T160829_20230725T162050_T17SLB,-82.611768,37.742717,1690302190026,None,None,None,None,None,None,None,None,None,None,ENOXY COAL INC
